In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from imblearn.under_sampling import NearMiss 
from sklearn.utils import resample
import pandas as pd
import h5py
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score,recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from collections import defaultdict
import scipy
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Applying Classification models

In [ ]:
models = []
models.append(('SVC-rbf', SVC(kernel = 'rbf', random_state = 5)))
models.append(('Logistic', LogisticRegression(random_state = 0)))
models.append(('KNN', KNeighborsClassifier(n_neighbors=13)))
models.append(('NaiveBayes', GaussianNB()))
models.append(('Random_Fores', RandomForestClassifier(n_estimators=5)))

scoring = 'accuracy'
predictions_dict=defaultdict(lambda : defaultdict(lambda : defaultdict(list)))                                               #to store confusion matrix and report

for band_no in tqdm(range(10)): 
  if(band_no%1 == 0):
    if band_no < 10:
      name = '%0*d' % (2,band_no+1)
    else:
      name = band_no+1
  file_x="/gdrive/MyDrive/Colab Notebooks/data/data_sub/entropies_band{}.csv".format(str(name))                                #X=bandwise entropies
  x=pd.read_csv(file_x)
  del(x[x.columns[0]])

  avg_acc=0
  predictions_final=0

  for label in tqdm(range(4)):
    file_y='/gdrive/MyDrive/Colab Notebooks/data/data_sub/label_th_3.5{}.h5'.format(str(label))                                #Y=oversampled label files
    print("Label:",label)
    with h5py.File(file_y, 'r') as hf1:
      y=hf1["y"][:]
    hf1.close()

    # split into 70:30 ration 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0) 

    X_train=X_train.fillna(0)
    X_test=X_test.fillna(0)

    X_train['y']=y_train
    majority = X_train[X_train.y == 1]
    minority = X_train[X_train.y == 0]

    oversample = resample(minority, replace=True, n_samples=len(majority), random_state=42)                                    #Oversampling
    # Returning to new training set
    oversample_train = pd.concat([majority, oversample])

    oversample_x_train = oversample_train.drop('y', axis=1)
    oversample_y_train = oversample_train.y

    predictions=np.array(np.arange(X_test.shape[0])).transpose()
    print(predictions.shape)
    for i in range(len(models)):
              name1, model=models[i]
              model.fit(oversample_x_train,oversample_y_train)
              y_pred=model.predict(X_test)
              accuracy=accuracy_score(y_test,y_pred)
              msg = "%s: %f" % (name1, accuracy)
              #print(msg) 
              predictions=np.vstack((predictions,y_pred))
              predictions_dict[band_no][label][name1]=[accuracy,classification_report(y_test, y_pred),confusion_matrix(y_test, y_pred)]
    predictions=predictions.transpose()[:,1:]

    y_result=scipy.stats.mode(predictions,axis=1)[0]                                                                          #mode of predicted y values
    predictions=np.hstack((predictions,y_result))
    print(accuracy_score(y_test,y_result))
    print(confusion_matrix(y_test,y_result))
    avg_acc+=(accuracy_score(y_test,y_result)*100)
    if label==0:
      predictions_final=predictions
    else:
      predictions_final=np.hstack((predictions_final,predictions))
  predictions_pd=pd.DataFrame(predictions_final)
  predictions_pd.to_csv("/gdrive/MyDrive/Colab Notebooks/data/data_sub/resampled_result_band{}.csv".format(str(name)))
  print('avg acc '+str(avg_acc/4))



  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]

Label: 0
(12288,)





 25%|██▌       | 1/4 [01:38<04:55, 98.64s/it]

0.653564453125
[[1152 1715]
 [2542 6879]]
Label: 1
(12288,)





 50%|█████     | 2/4 [03:12<03:14, 97.20s/it]

...

0.641357421875
[[1226 1732]
 [2675 6655]]
Label: 2
(12288,)





 75%|███████▌  | 3/4 [04:36<01:32, 92.31s/it]

0.6996256510416666
[[ 992 1746]
 [1945 7605]]
Label: 3
(12288,)





100%|██████████| 4/4 [06:11<00:00, 92.77s/it]


100%|██████████| 10/10 [1:05:13<00:00, 391.33s/it]

0.5309244791666666
[[1515 1225]
 [4539 5009]]
avg acc 65.60262044270833


In [ ]:
delete=predictions_dict

In [ ]:
import json

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

json.dump(delete,open('/gdrive/MyDrive/Colab Notebooks/data/data_sub/resampled_result_dict.json','w'),cls=NumpyEncoder)         #stores predicted y values for all models and their mode

In [ ]:
from scipy import stats

Choosing top 3 models and creating fusion of those models

In [ ]:
for i1 in range(10):
  if(i1%1 == 0):
    if i1 < 10:
      name = '%0*d' % (2,i1+1)
    else:
      name = i1+1
  file_x="/gdrive/MyDrive/Colab Notebooks/data/data_sub/entropies_band{}.csv".format(str(name))
  x=pd.read_csv(file_x)
  del(x[x.columns[0]])

  band_pred=pd.read_csv("/gdrive/MyDrive/Colab Notebooks/data/data_sub/resampled_result_band{}.csv".format(str(name)))
  del(band_pred[band_pred.columns[0]])

  knn_pred=band_pred[band_pred.columns[2:23:6]]
  random_pred=band_pred[band_pred.columns[4:23:6]]
  svc_pred=band_pred[band_pred.columns[0:23:6]]

  # mode_res=np.array(np.arange(knn_pred.shape[0])).transpose()
  print("Band:", name)

  for label in range(4):
    file_y='/gdrive/MyDrive/Colab Notebooks/data/data_sub/label_th_3.5{}.h5'.format(str(label))
    print("Label:",label)
    with h5py.File(file_y, 'r') as hf1:
      y=hf1["y"][:]
    hf1.close()

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0) 
    
    mode_res=stats.mode([knn_pred[knn_pred.columns[i]],svc_pred[svc_pred.columns[i]],random_pred[random_pred.columns[i]]])
    # mode_res=np.vstack((mode_res,temp))
    print("accuracy score",accuracy_score(y_test,mode_res[0].transpose()))
  

Band: 01
Label: 0
accuracy score 0.7071940104166666
Label: 1
accuracy score 0.572998046875
Label: 2
accuracy score 0.6478678385416666
Label: 3
accuracy score 0.736083984375
Band: 02
Label: 0
accuracy score 0.6923014322916666
Label: 1
accuracy score 0.57275390625
Label: 2
accuracy score 0.6355794270833334
Label: 3
accuracy score 0.7263997395833334
Band: 03
Label: 0
accuracy score 0.7096354166666666
Label: 1
accuracy score 0.5777180989583334
Label: 2
accuracy score 0.6512858072916666
Label: 3
accuracy score 0.7427571614583334
Band: 04
Label: 0
accuracy score 0.7035319010416666
Label: 1
accuracy score 0.576171875
Label: 2
accuracy score 0.6468098958333334
Label: 3
accuracy score 0.73291015625
Band: 05
Label: 0
accuracy score 0.7097981770833334
Label: 1
accuracy score 0.5799967447916666
Label: 2
accuracy score 0.6488444010416666
Label: 3
accuracy score 0.7354329427083334
Band: 06
Label: 0
accuracy score 0.6878255208333334
Label: 1
accuracy score 0.5668131510416666
Label: 2
accuracy score 0